# Import Libraries 

In [71]:
import pandas as pd
import numpy as np

# Define functions

In [101]:
def preprocess_raw_data(df):
    
    for col in df.columns:
#         df[col] = df[col].str.replace('"', '', regex=True)
        df[col] = df[col].str.replace('{', '', regex=True)
        df[col] = df[col].str.replace('}', '', regex=True)

    return df

def unique_keys(df, col):
    unq = []
    for row in range(len(df)):
        split_1 = len(df[col].iloc[row].split(':')) - 1
        
        for i in range(split_1):
            
            last_element = len(df[col].iloc[row].split(':')[i].split(',')) - 1
            unq.append(df[col].iloc[row].split(':')[i].split(',')[last_element])            
            
    print(np.unique(unq))

# Read data

In [102]:
xls = pd.ExcelFile('data/raw.xlsx')

In [103]:
xls.sheet_names

['ANAMET', 'Sheet6', 'UNS', 'ISRI', 'Materials', 'Process']

## UNS

In [104]:
uns = pd.read_excel(xls, 'UNS')
print(uns.shape)

(8261, 4)


In [105]:
uns = preprocess_raw_data(uns)
uns.head()

,Source Entity,Target Entity,Relationship,Unnamed: 3
0,"""name"":C83300,""full_community_id"":712","""MeltingPoint"":327.5,""Ar"":207.2,""Symbol"":Pb,""n...",IS_COMPOSED_OF,NaN
1,"""name"":C83300,""full_community_id"":712","""MeltingPoint"":231.9,""Ar"":118.71,""Symbol"":Sn,""...",IS_COMPOSED_OF,NaN
2,"""name"":C83300,""full_community_id"":712","""MeltingPoint"":1085,""Ar"":63.546,""Symbol"":Cu,""n...",IS_COMPOSED_OF,NaN
3,"""name"":C83300,""full_community_id"":712","""MeltingPoint"":419.5,""Ar"":65.38,""Symbol"":Zn,""n...",IS_COMPOSED_OF,NaN
4,"""name"":C83300,""full_community_id"":712","""MeltingPoint"":419.5,""Ar"":65.38,""Symbol"":Zn,""n...",IS_COMPOSED_OF,NaN


In [106]:
unique_keys(uns, 'Source Entity')

['"full_community_id"' '"name"']


In [107]:
def full_community_id_and_name(df):
    
    df['name'] = np.nan
    df['full_community_id'] = np.nan

    for row in range(len(df)):
        df['name'].iloc[row] = df['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
        df['full_community_id'].iloc[row] = df['Source Entity'].str.split(',').iloc[row][1].split(':')[1]

    df.drop(columns='Source Entity', inplace=True)
    return df

In [108]:
uns['name'] = np.nan
uns['full_community_id'] = np.nan

for row in range(len(uns)):
    uns['name'].iloc[row] = uns['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
    uns['full_community_id'].iloc[row] = uns['Source Entity'].str.split(',').iloc[row][1].split(':')[1]

uns.drop(columns='Source Entity', inplace=True)
uns.head()

/tmp/ipykernel_4525/2960562481.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uns['name'].iloc[row] = uns['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
/tmp/ipykernel_4525/2960562481.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uns['full_community_id'].iloc[row] = uns['Source Entity'].str.split(',').iloc[row][0].split(':')[1]


IndexError: list index out of range

### Create `full_community_id` and `name` columns

In [ ]:
materials['name'] = np.nan
materials['full_community_id'] = np.nan

for row in range(len(materials)):
    materials['name'].iloc[row] = materials['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
    materials['full_community_id'].iloc[row] = materials['Source Entity'].str.split(',').iloc[row][1].split(':')[1]

materials.drop(columns='Source Entity', inplace=True)
materials.head()

## MATERIALS

In [52]:
materials = pd.read_excel(xls, 'Materials')
print(materials.shape)

(364, 3)


In [53]:
materials = preprocess_raw_data(materials)
materials.head()

,Source Entity,Target Entity,Relationship
0,"name:ALUMINUM,full_community_id:244","Free_of:paper, plastic, ink, and any other con...",IS_COMPOSED_OF
1,"name:ALUMINUM,full_community_id:244","Free_of:paper, plastic, excessively inked shee...",IS_COMPOSED_OF
2,"name:ALUMINUM,full_community_id:244","Free_of:Radioactive material, dross, or ash,Ca...",IS_COMPOSED_OF
3,"name:ALUMINUM,full_community_id:244","Free_of:Radar chaff foil, chemically etched fo...",IS_COMPOSED_OF
4,"name:ALUMINUM,full_community_id:244","Free_of:Hair wire, wire screen, dirt and other...",IS_COMPOSED_OF


In [54]:
new_cols = []
for row in range(len(materials)):
    
    split_1_dimensions = len(materials['Source Entity'].iloc[row].split(':')) - 1
    
    for i in range(split_1_dimensions):
        
        split_2_dimensions = len(materials['Source Entity'].iloc[row].split(':')[i].split(','))
 
        last_element = len(materials['Source Entity'].iloc[row].split(':')[i].split(','))-1
        new_cols.append(materials['Source Entity'].iloc[row].split(':')[i].split(',')[last_element])

In [55]:
np.unique(new_cols)

array(['full_community_id', 'name'], dtype='<U17')

In [56]:
materials['name'] = np.nan
materials['full_community_id'] = np.nan

for row in range(len(materials)):
    materials['name'].iloc[row] = materials['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
    materials['full_community_id'].iloc[row] = materials['Source Entity'].str.split(',').iloc[row][1].split(':')[1]

materials.drop(columns='Source Entity', inplace=True)
materials.head()

/tmp/ipykernel_4525/3151878116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  materials['name'].iloc[row] = materials['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
/tmp/ipykernel_4525/3151878116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  materials['full_community_id'].iloc[row] = materials['Source Entity'].str.split(',').iloc[row][1].split(':')[1]


,Target Entity,Relationship,name,full_community_id
0,"Free_of:paper, plastic, ink, and any other con...",IS_COMPOSED_OF,ALUMINUM,244
1,"Free_of:paper, plastic, excessively inked shee...",IS_COMPOSED_OF,ALUMINUM,244
2,"Free_of:Radioactive material, dross, or ash,Ca...",IS_COMPOSED_OF,ALUMINUM,244
3,"Free_of:Radar chaff foil, chemically etched fo...",IS_COMPOSED_OF,ALUMINUM,244
4,"Free_of:Hair wire, wire screen, dirt and other...",IS_COMPOSED_OF,ALUMINUM,244


In [58]:
new_cols = []
for row in range(len(materials)):
    
    split_1_dimensions = len(materials['Target Entity'].iloc[row].split(':')) - 1
    
    for i in range(split_1_dimensions):
        
        split_2_dimensions = len(materials['Target Entity'].iloc[row].split(':')[i].split(','))
 
        last_element = len(materials['Target Entity'].iloc[row].split(':')[i].split(','))-1
        new_cols.append(materials['Target Entity'].iloc[row].split(':')[i].split(',')[last_element])

In [59]:
np.unique(new_cols)

array(['Ar', 'AtomicNumber', 'Categorisation', 'Density', 'Dimensions',
       'Form', 'Free_of', 'Material', 'MeltingPoint', 'Series',
       'Shall be made up of a combination of the nonferrous metals',
       'Specifications', 'Symbol', 'Title', 'full_community_id', 'name'],
      dtype='<U58')

#### `Shall be made up of a combination of the nonferrous metals` should not be a column but an element

In [62]:
new_cols.remove('Shall be made up of a combination of the nonferrous metals')

ValueError: list.remove(x): x not in list

In [64]:
np.unique(new_cols)

array(['Ar', 'AtomicNumber', 'Categorisation', 'Density', 'Dimensions',
       'Form', 'Free_of', 'Material', 'MeltingPoint', 'Series',
       'Specifications', 'Symbol', 'Title', 'full_community_id', 'name'],
      dtype='<U17')

In [70]:
for row in range(len(materials)):
    
    split_1_dimensions = len(materials['Target Entity'].iloc[row].split(':')) - 1
    
    for i in range(split_1_dimensions):
        
        split_2_dimensions = len(materials['Target Entity'].iloc[row].split(':')[i].split(','))
 
        last_element = len(materials['Target Entity'].iloc[row].split(':')[i].split(','))-1
        materials['Target Entity'].iloc[row].split(':')[i].split(',')[last_element])

['Free_of']
['paper', ' plastic', ' ink', ' and any other contaminants', 'Categorisation']
['Nonferrous Scrap - Aluminum', 'Specifications']
['New', ' clean. To consist of 1000 and/or 3000 series alloys', ' uncoated', ' unpainted', 'Form']
['Lithographic Sheets', 'name']
['Tabloid', 'Dimensions']
['Minimum size of 3� (8 cm) in any direction', 'Title']
['New', ' Clean Aluminum Lithographic Sheets', 'Material']
['Aluminum', 'full_community_id']
['Free_of']
['paper', ' plastic', ' excessively inked sheets', ' and any other contaminants', 'Categorisation']
['Nonferrous Scrap - Aluminum', 'Specifications']
['Clean. To consist of 1000 and/or 3000 series alloys', 'Form']
['Lithographic Sheets', 'name']
['Tablet', 'Title']
['Clean Aluminum Lithographic Sheets', 'Material']
['Aluminum ', 'full_community_id']
['Free_of']
['Radioactive material', ' dross', ' or ash', 'Categorisation']
['Nonferrous Scrap - Aluminum', 'Specifications']
['Shall be made up of a combination of the nonferrous metals']


In [32]:
materials['Target Entity'].iloc[2]

'Free_of:Radioactive material, dross, or ash,Categorisation:Nonferrous Scrap - Aluminum,Specifications:Shall be made up of a combination of the nonferrous metals: aluminum, copper, lead, magnesium, stainless steel, nickel, tin, and zinc, in elemental or alloyed (solid) form. Material generated by eddy current, air separation, flotation, screening, other segregation technique(s), or a combination thereof. Shall have passed one or more magnets to reduce or eliminate free iron and/or large iron attachments. Material to be bought/sold under this guideline shall be identified as �Zorba� with a number to follow indicating the estimated percentage nonferrous metal content of the material (e.g., �Zorba 90� means the material contains approximately 90% nonferrous metal content). May also be screened to permit description by specific size ranges. (Refer also to Zorba under Mixed Metals.),Form:Nonferrous Scrap,name:Zorba,Title:Shredded Nonferrous Scrap (predominantly aluminum),full_community_id:9

In [ ]:
new_cols_df = pd.DataFrame(new_cols)
new_cols_df.value_counts()

In [ ]:
new_cols_df[new_cols_df[0] == 'Shall be made up of a combination of the nonferrous metals']

In [ ]:
materials['Target Entity'].iloc[0].split(':')[1].split(',')[len(materials['Target Entity'].iloc[0].split(':')[1].split(','))-1]

## ANAMET

In [40]:
anamet = pd.read_excel(xls, 'ANAMET')
display(anamet.head(3))
print(anamet.shape)

,Node ID,Source Entity,Target Entity,Relationship,Node ID.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,437.0,"{""Description"":ΣΚΡΑΠ ΨΑΛΙΔΙΣΜΕΝΟ ΠΥΚΝ >450 ΚG/...","{""Description"":Σύρματα + διάφορο,""Category"":Fe...",CONTAINS,437.0,NaN,NaN,NaN,NaN,NaN,NaN
1,437.0,"{""Description"":ΣΚΡΑΠ ΨΑΛΙΔΙΣΜΕΝΟ ΠΥΚΝ >450 ΚG/...","{""Description"":Βαρύ για ΣΟΒΕΛ,""Category"":Ferro...",CONTAINS,437.0,NaN,NaN,NaN,NaN,NaN,NaN
2,437.0,"{""Description"":ΣΚΡΑΠ ΨΑΛΙΔΙΣΜΕΝΟ ΠΥΚΝ >450 ΚG/...","{""Description"":ζάντες + βαρύ προς shredder κλπ...",CONTAINS,437.0,NaN,NaN,NaN,NaN,NaN,NaN


(1235, 11)


# Process Sheet

In [41]:
process = pd.read_excel(xls, 'Process')
display(process.head(3))
print(process.shape)

,Source Entity,Target Entity,Relationship
0,"{""name"":Pyrometallurgical,""full_community_id"":...","{""Free_of"":semi-red brass castings (78% to 81%...",IS_PROCESSED_WITH
1,"{""name"":Pyrometallurgical,""full_community_id"":...","{""Categorisation"":Nonferrous Scrap - Red Metal...",IS_PROCESSED_WITH
2,"{""name"":Pyrometallurgical,""full_community_id"":...","{""Free_of"":brass forgings, silicon bronze, alu...",IS_PROCESSED_WITH


(105, 3)


In [42]:
process = preprocess_raw_data(process)
process.head()

,Source Entity,Target Entity,Relationship
0,"name:Pyrometallurgical,full_community_id:257",Free_of:semi-red brass castings (78% to 81% co...,IS_PROCESSED_WITH
1,"name:Pyrometallurgical,full_community_id:257","Categorisation:Nonferrous Scrap - Red Metals,S...",IS_PROCESSED_WITH
2,"name:Pyrometallurgical,full_community_id:257","Free_of:brass forgings, silicon bronze, alumin...",IS_PROCESSED_WITH
3,"name:Pyrometallurgical,full_community_id:257","Categorisation:Nonferrous Scrap - Red Metals,S...",IS_PROCESSED_WITH
4,"name:Pyrometallurgical,full_community_id:257","Free_of:Yellow boxes and iron-backed boxes,Cat...",IS_PROCESSED_WITH


In [43]:
process['name'] = np.nan
process['full_community_id'] = np.nanNo

In [44]:
for row in range(len(process)):
    process['name'].iloc[row] = process['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
    process['full_community_id'].iloc[row] = process['Source Entity'].str.split(',').iloc[row][1].split(':')[1]

process.drop(columns='Source Entity', inplace=True)
process.head()

/tmp/ipykernel_4525/2360545996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  process['name'].iloc[row] = process['Source Entity'].str.split(',').iloc[row][0].split(':')[1]
/tmp/ipykernel_4525/2360545996.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  process['full_community_id'].iloc[row] = process['Source Entity'].str.split(',').iloc[row][1].split(':')[1]


,Target Entity,Relationship,name,full_community_id
0,Free_of:semi-red brass castings (78% to 81% co...,IS_PROCESSED_WITH,Pyrometallurgical,257
1,"Categorisation:Nonferrous Scrap - Red Metals,S...",IS_PROCESSED_WITH,Pyrometallurgical,257
2,"Free_of:brass forgings, silicon bronze, alumin...",IS_PROCESSED_WITH,Pyrometallurgical,257
3,"Categorisation:Nonferrous Scrap - Red Metals,S...",IS_PROCESSED_WITH,Pyrometallurgical,257
4,"Free_of:Yellow boxes and iron-backed boxes,Cat...",IS_PROCESSED_WITH,Pyrometallurgical,257


In [45]:
new_cols = []
for row in range(len(process)):
    
    split_1_dimensions = len(process['Target Entity'].iloc[row].split(':')) - 1
    
    for i in range(split_1_dimensions):
        
#         split_2_dimensions = len(process['Target Entity'].iloc[row].split(':')[i].split(','))
 
        last_element = len(process['Target Entity'].iloc[row].split(':')[i].split(','))-1
        new_cols.append(process['Target Entity'].iloc[row].split(':')[i].split(',')[last_element])

In [46]:
process['Target Entity'].iloc[12].split(':')[1].split(',')

['bullets', ' iron and any other foreign material', 'Categorisation']

In [47]:
pd.DataFrame(new_cols).value_counts()

full_community_id                                                                105
name                                                                             105
Categorisation                                                                    54
Form                                                                              54
Title                                                                             54
Specifications                                                                    52
Category                                                                          51
Material                                                                          46
Free_of                                                                           39
Description                                                                       36
EDescription                                                                      15
GDescription                                                     

In [ ]:
process['name'].value_counts()

In [ ]:
process['Relationship'].value_counts()